In [40]:
import folium
import json
import pandas as pd
from google.colab import files
import requests
from urllib.parse import quote
from IPython.display import display

In [43]:
# 행안부 API 키 가져오기
uploaded = files.upload()
filename = list(uploaded.keys())[0]
with open(filename) as f:
    r_api_key = f.read()

Saving road_api_key.txt to road_api_key.txt


In [44]:
# 카카오 API 키 가져오기
uploaded = files.upload()
filename = list(uploaded.keys())[0]
with open(filename) as f:
    k_api_key = f.read()

Saving 카톡_api_key.txt to 카톡_api_key.txt


In [45]:
# 데이터 프레임
df = pd.DataFrame(columns = ['장소', '도로명주소', '위도', '경도'])

In [ ]:
i = 0
# 입력값 받기
bldg = input('지도에 표시하고 싶은 장소를 입력하세요 : ')
while bldg.lower() != 'q':
    base_url = "https://business.juso.go.kr/addrlink/addrLinkApi.do"
    params1 = f'currentaPage=1&countPerPage=10&confmKey={r_api_key}'
    params2 = f'keyword={quote(bldg)}&resultType=json'
    url = f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    res = json.loads(result.text)
    try:
        name = res['results']['juso'][0]['bdNm'] # 장소명
        road_add = res['results']['juso'][0]['roadAddr'] #도로주소명
        search_url = "https://dapi.kakao.com/v2/local/search/address.json"
        url = f'{search_url}?query={quote(road_add)}'
        result = requests.get(url, headers={"Authorization": f"KakaoAK {k_api_key}"}).json()
        lng = float(result['documents'][0]['x']) # 경도
        lat = float(result['documents'][0]['y']) # 위도
        # 데이터 프레임에 추가
        df.loc[i] = [name, road_add, lat, lng]
        display(df)
    except:
        print('잘못 입력하셨습니다.')
    print('-------------------------------------------------------------------------------------')
    bldg = input('지도에 표시하고 싶은 장소를 입력하세요\n - 그만 입력하시려면 q를 입력\n - 방금 적은 항목을 삭제하려면 d를 입력 : ')
    try:
        if bldg.lower() == 'd': # 최근 추가 항목 삭제하기
            df = df.drop([i])
            display(df)
            print('-------------------------------------------------------------------------------------')
            bldg = input('지도에 표시하고 싶은 장소를 입력하세요\n - 그만 입력하시려면 q를 입력\n - 방금 적은 항목을 삭제하려면 d를 입력 : ')
            i += 1
    except:
        print('더 이상 삭제할 항목이 없습니다.')
        print('-------------------------------------------------------------------------------------')
        bldg = input('지도에 표시하고 싶은 장소를 입력하세요\n - 그만 입력하시려면 q를 입력\n - 방금 적은 항목을 삭제하려면 d를 입력 : ')
df # 최종 데이터 프레임 출력

In [ ]:
# 지도에 표시하기
map = folium.Map(location=[37.55, 126.98], zoom_start=10) # 지도 만들기
for name, addr, lat, lng in zip(df.장소, df.도로명주소, df.위도, df.경도):
    folium.Marker([lat, lng], 
                  tooltip = name, # 장소명
                  popup = addr # 도로주소명
    ).add_to(map)
map

In [ ]:
# 만약 동명의 장소가 여러개 나온다면 ->
# 장소 이름들을 리스트로 출력 후 선택하게 하고
# 그 장소의 대한 정보를 가져오기